In [ ]:
from fenics import *
import numpy as np
from braininversion.IOHandling import read_mesh_from_h5, write_to_xdmf, xdmf_to_unstructuredGrid, read_xdmf_timeseries
from fenics_adjoint import *
import moola
from braininversion.DarcySolver import solve_darcy
from dolfin.cpp.log import info
from braininversion.Optimization import optimize_darcy_source, compute_minimization_target
from braininversion.reconstructByLaplaceSolve import(solve_laplace_for_all_timesteps,
                                     compute_sources_from_pressures)
import pyvista as pv

set_log_level(40)
parameters['ghost_mode'] = 'shared_facet' 

# Form compiler options
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True

h5_filename = "../meshes/colin27_coarse_boundaries.h5"
outfile = "../results/laplace_opt_darcy_colins27_coarse.xdmf"

# time stepping
T = 2.0           # final time
num_steps = 50    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4          # storage coefficient
mmHg2Pa = 132.32
material_parameter = {"K":K, "c":c}

mesh, boundary_marker = read_mesh_from_h5(h5_filename) #skull: 1, ventricle:2

f = 1
A = 0.2*mmHg2Pa
p_skull = Expression("A*sin(2*pi*f*t)", A=A,f=f,t=0,degree=2)
p_ventricle = Expression("A*sin(2*pi*f*t)", A=A*0.5,f=f,t=0,degree=2)
#p_ventricle = Constant(0.0)


boundary_conditions = {1:{"Neumann":Constant(0.0)},
                       2:{"Neumann":Constant(0.0)}}

                        
time_dep_expr = [p_skull, p_ventricle]

laplace_bc = {2:{"Dirichlet":p_ventricle}, 1:{"Dirichlet":p_skull}}

In [ ]:
laplace_pressures = solve_laplace_for_all_timesteps(mesh, boundary_marker, laplace_bc, 
                                                   times, time_dep_expr)

laplace_sources = compute_sources_from_pressures(laplace_pressures, c, dt)

laplace_solution = solve_darcy(mesh, laplace_sources, T, num_steps, K,
                              boundary_marker, boundary_conditions,
                              c=c)
laplace_solution = [s.copy() for s in laplace_solution]
[p.rename("pressure","pressure") for p in laplace_solution]
[g.rename("source","source") for g in laplace_sources]

write_to_xdmf([laplace_solution, laplace_sources],
              outfile, times)

In [ ]:
grid = xdmf_to_unstructuredGrid(outfile, idx=[0,1])


In [ ]:
slices = grid.slice_orthogonal()
p = pv.Plotter()
p.add_mesh(slices,cmap="coolwarm")
p.show()

In [ ]:
clipped_data = grid.clip(normal=[0,0,1], value=0.0)
p = pv.PlotterITK()
p.add_mesh(clipped_data)
p.show()